# Preprocesamiento de datos

En este documento veremos algunos ejemplos de preprocesamiento de datos 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer, MinMaxScaler

Haremos uso de la base de datos ejemplo de iris. Parte de los datasets de scikitlearn
Mostramos la descripción de la base de datos asi como una vista con pandas de los datos originales.


In [ ]:
data = load_iris()
X = data.data  
Y = data.target  

In [ ]:
print(data.DESCR)

In [ ]:
df = pd.DataFrame(data= data.data, columns= data['feature_names'] )
df

## Tratamiento de duplicados 
Pandas ofrece una funcionalidad especifica para encontrar elementos duplicados, esto puede ser sobre todas las columnas o solo sobre un conjunto, también es útil para comprobar identificadores.


In [ ]:
duplicados = df.duplicated(keep=False)
df[duplicados]

## Separación de entrenamiento
Sklearn puede separar la base datos de manera aleatoria en entrenamiento y prueba.


In [ ]:
X_train, X_test = train_test_split(X)

Podemos ver como quedan las tablas una vez que han quedado divididas.

In [ ]:
pd.DataFrame(
    data= X_train, 
    columns= data['feature_names'] 
    )

In [ ]:
pd.DataFrame(
    data= X_test, 
    columns= data['feature_names'] 
    )

## Estandarizado / escalado
Tenemos varios métodos para estandarizar los datos.
El siguiente método es el estándar el cual lleva los datos a una distribución normal estándar. Por lo cual la media es 0 y varianza es 1 


In [ ]:
scaler_estandar = StandardScaler().fit(X_train)

In [ ]:
df2 = pd.DataFrame(
    data= scaler_estandar.transform(X_train), 
    columns= data['feature_names'] 
    )
df2

In [ ]:
df2.std()

In [ ]:
df2.mean()

Es importante conservar el transformador de escala ya que ese debe ser fijado y utilizado para ambos conjuntos.

In [ ]:
pd.DataFrame(
    data= scaler_estandar.transform(X_test), 
    columns= data['feature_names'] 
    )

También está disponible el estandarizado minmax el cual lleva todos los datos al rango de 0 a 1 donde 0 es el valor mínimo y 1 el máximo. 

In [ ]:
scaler_minmax = MinMaxScaler().fit(X)

In [ ]:
df3 = pd.DataFrame(
    data= scaler_minmax.transform(X_train), 
    columns= data['feature_names'] )
df3

In [ ]:
df3.min()

In [ ]:
df3.max()

**Ejercicio:** aplicar esta transformación al conjunto de prueba y calcular el mínimo y el máximo

## Reducción de dimensionalidad
Vemos un ejemplo de aplicación de componentes principales, a pesar de tener una cantidad reducida de variables.



In [ ]:
pd.DataFrame(
    data= scaler_minmax.transform(X_test), 
    columns= data['feature_names'] )

In [ ]:
pca = PCA(n_components=2) 
X_pca = pca.fit_transform(X_test)

In [ ]:
print("Varianza explicada por cada componente principal:")
print(pca.explained_variance_ratio_)

Encontramos que la 97% de la variabilidad queda explicada por dos componentes 

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

## Creación de características
Podemos crear todos los cruces de las características así como los cuadrados de estas con ayuda de sklearn


In [ ]:
poly = PolynomialFeatures(2)

pd.DataFrame(
    data= poly.fit_transform(X_train), 
    columns= poly.get_feature_names_out(data['feature_names']))


También podemos definir transformaciones, por ejemplo aplicar el logaritmo a las variables 

In [ ]:
transformer = FunctionTransformer(np.log1p, validate=True)

In [ ]:
pd.DataFrame(
    data= transformer.transform(X_train), 
    columns= data['feature_names'] )


## Integración de múltiples fuentes
Para esto tomaremos la muestra del censo de población y vivienda 2020 con las tabas de personas y viviendas.


In [ ]:
personas = pd.read_csv("https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CPV_CB_Personas_ejemplo_csv.zip",dtype=str)
personas

In [ ]:
viviendas = pd.read_csv("https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CPV_CB_Viviendas_ejemplo_csv.zip",dtype=str)
viviendas



Las unimos asegurándonos que cada vivienda tenga una o mas personas, es muy importante revisar los conteos.

In [ ]:
unido = pd.merge(
    viviendas,
    personas,
    on = "ID_VIV",
    how="inner",
    validate="one_to_many"
)
unido